In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path='/content/drive/Shareddrives/Multicampus/mimic-iii-clinical-database-1.4/'

In [ ]:
icd = pd.read_csv(data_path+'DIAGNOSES_ICD.csv.gz')

In [ ]:
icd.columns

In [ ]:
icd.head()

In [ ]:
icd.info()  #SEQ_NUM과 ICD_CODE에 47개의 결측값 있음

In [ ]:
icd['ICD9_CODE']  #숫자와 문자 섞인 상태(앞자리에 알파벳 포함)

In [ ]:
icd['ICD9_CODE'].value_counts().count()  #6984개의 ICD9_CODE 사용됨

001–139: infectious and parasitic diseases(전염성 및 기생충 질환)

140–239: neoplasms(미상의 새로운 미생물)

240–279: endocrine, nutritional and metabolic diseases, and immunity disorders(내분비, 영양 및 대사 질환, 면역 장애)

280–289: diseases of the blood and blood-forming organs(혈액 및 조형기관 질병)

290–319: mental disorders(정신장애)

320–389: diseases of the nervous system and sense organs(신경계 및 감각기관 질병)

390–459: diseases of the circulatory system(순환계 질환)

460–519: diseases of the respiratory system(호흡기 질환)

520–579: diseases of the digestive system(소화기 질환)

580–629: diseases of the genitourinary system(비뇨기 질환)

630–679: complications of pregnancy, childbirth, and the puerperium(임신, 출산 및 관련 합병증)

680–709: diseases of the skin and subcutaneous tissue(피부 및 피하조직 질환)

710–739: diseases of the musculoskeletal system and connective tissue(근골격계 및 결합조직 질병)

740–759: congenital anomalies(선천성 기형)

760–779: certain conditions originating in the perinatal period(출산 전후에 발생하는 특정 상태)

780–799: symptoms, signs, and ill-defined conditions(증상, 징후만 있는 불분명한 상태)

800–999: injury and poisoning(부상 및 중독)


In [ ]:
icd['new_code'] = icd['ICD9_CODE']

In [ ]:
icd['new_code'][~icd['new_code'].str.contains("[a-zA-Z]").fillna(False)]

In [ ]:
icd['new_code'] = icd['new_code'][~icd['new_code'].str.contains("[a-zA-Z]").fillna(False)]
icd['new_code']

In [ ]:
icd['new_code'] = icd['new_code'].str[0:3]
icd['new_code']

In [ ]:
icd['new_code']=icd['new_code'].fillna(value=999)
# E나 V가 앞에 달리는 경우, 부상의 외부 원인 및 추가분류한 것이다.
# 이는 999 표시자로 사용하기로 되어 있음
# 800~999 코드가 부상 및 중독과 관련된 코드이므로
icd['new_code']

In [ ]:
icd['new_code'] = icd['new_code'].astype(int)
icd['new_code']

In [ ]:
# 위에서 언급했던 코드 범위대로 구간을 나눈다
icd9_ranges = [(1, 140), (140, 240), (240, 280), (280, 290), (290, 320), (320, 390), 
               (390, 460), (460, 520), (520, 580), (580, 630), (630, 680), (680, 710),
               (710, 740), (740, 760), (760, 780), (780, 800), (800, 1000)]   

In [ ]:
diag_dict = {0: 'infectious', 1: 'neoplasms', 2: 'endocrine', 3: 'blood',
             4: 'mental', 5: 'nervous', 6: 'circulatory', 7: 'respiratory',
             8: 'digestive', 9: 'genitourinary', 10: 'pregnancy', 11: 'skin', 
             12: 'muscular', 13: 'congenital', 14: 'prenatal', 15: 'misc',
             16: 'injury'}

In [ ]:
for idx, rng in enumerate(icd9_ranges):
    icd['new_code'] = np.where(icd['new_code'].between(rng[0],rng[1]),
                               idx,
                               icd['new_code'])

In [ ]:
list(enumerate(icd9_ranges))

In [ ]:
icd['new_code'].value_counts()

In [ ]:
icd['diag'] = icd['new_code'].replace(diag_dict)

In [ ]:
icd.head()

In [ ]:
icd.columns

In [ ]:
diag_list = icd.groupby('SUBJECT_ID')['diag'].apply(list).reset_index()
diag_list.head()

In [ ]:
diag_items = pd.get_dummies(diag_list['diag'].apply(pd.Series).stack()).sum(level=0)

In [ ]:
diag_items.head()

In [ ]:
# diag_items = diag_items.join(diag_list['SUBJECT_ID'], how="outer")
icd9 = diag_list.join(diag_items, how="outer")

In [ ]:
icd9.head()

In [ ]:
newcode_list = icd.groupby('SUBJECT_ID')['new_code'].apply(list).reset_index()
newcode_list

In [ ]:
icd9 = newcode_list.merge(icd9, on='SUBJECT_ID',how='inner')
icd9.head()

In [ ]:
del icd9['diag']

In [ ]:
icd9.head()

In [ ]:
len(icd9['SUBJECT_ID'].unique())

# 중복되는 subject_ID 제거

In [ ]:
admissions = pd.read_csv(data_path+'ADMISSIONS.csv.gz')

In [ ]:
admissions_nodup=admissions.drop_duplicates(['SUBJECT_ID'], keep=False)

In [ ]:
len(admissions_nodup['SUBJECT_ID'])

In [ ]:
icd9 = icd9.merge(admissions_nodup['SUBJECT_ID'], on='SUBJECT_ID',how='inner')
icd9.head()

In [ ]:
len(icd9['SUBJECT_ID'])

In [ ]:
icd9.to_csv("icd9_prepro.csv", index=False)

# 이상치 및 결측값 확인

진단받은 질병에 대한 분류이므로 이상치로 보일 수 있는 값은 없다

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Multicampus/Preprocessing/icd9_prepro.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df[df.sum(axis=1) <= 0]  # 결측값 없음

In [ ]:
df.iloc[:,2:].sum()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,12))
df.iloc[:,2:].sum().plot(kind='bar')